# 00 — Quickstart

**Goal:** Verify env + tiny text generation + mini RAG.

**Inputs:** Prompt text; local docs

**Outputs:** Generated text; top-matching docs

> Tip: Fill your API keys in `.env` (copy from `.env.example`). Never commit secrets.


In [ ]:
import sys, platform
print('Python:', sys.version) ; print('Platform:', platform.platform())

### Tiny text generation example (CPU-friendly)

In [ ]:
from transformers import pipeline
pipe = pipeline('text-generation', model='sshleifer/tiny-gpt2')
out = pipe("Say hello from Project Pythia in one sentence.", max_new_tokens=20)
print(out[0]['generated_text'])


### Mini RAG using TF‑IDF on docs/

In [ ]:
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

files = sorted(glob.glob('../docs/**/*.md', recursive=True))
texts, paths = [], []
for p in files:
    with open(p, 'r', encoding='utf-8') as f:
        texts.append(f.read())
        paths.append(p)

if not texts:
    print("No docs found under ../docs")
else:
    vec = TfidfVectorizer(stop_words='english')
    X = vec.fit_transform(texts)
    def search(q, k=3):
        qv = vec.transform([q])
        sims = cosine_similarity(qv, X).ravel()
        idxs = sims.argsort()[::-1][:k]
        return [(paths[i], float(sims[i])) for i in idxs]
    print(search("What is this cookbook about?"))
